In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

# (issue) with procedures table

* If ex. 4333 is in both dictionarys with procedure and diagnose label this will either be repeated or overwritten
* 331 is an example of a code repeated in procedure dictionary with different labels, so again wil both be repreated or overwritten

In [ ]:
df_proc = pd.read_csv('mimic-iii-clinical-database-1.4/PROCEDURES_ICD.csv.gz')
df_diag  = pd.read_csv('mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv.gz') 

In [ ]:
df_proc.head()

In [ ]:
df_diag.head()

# label the proedures with help of ICD9_CODES from bioportal

some of the codes in procedure table contain diseases codes as from the website

https://www.findacode.com/search/search.php?str=3601

we can find the codes and se that exp. 4441 is both in procedures and in diseases, but 360.1 is only in diseases which we have in the procedures table

also 374 is in different catgeory both in procedure and in disease depending on which volume of the icd9 is used

In [ ]:
ICD9_CODES = pd.read_csv('ICD9_PROC_CODE.csv')
ICD9_CODES = ICD9_CODES.astype(str)

ICD9_CODES.head()

In [ ]:
ICD9_CODES[ICD9_CODES['0'] == '360.1']

In [ ]:
ICD9_LABEL = {
    '001-999.99': 'DISEASES AND INJURIES',
    '00-99.99': 'PROCEDURES',
    'E000-E999.9':'SUPPLEMENTARY CLASSIFICATION OF EXTERNAL CAUSES OF INJURY AND POISONING',
    'V01-V91.99':'SUPPLEMENTARY CLASSIFICATION OF FACTORS INFLUENCING HEALTH STATUS AND CONTACT WITH HEALTH SERVICES',
    'T051': 'Event',
    'T071': 'Entity'}

In [ ]:
def getcode(code):
    # find row 
    row = ICD9_CODES[ICD9_CODES['0'].str.replace(".","") == str(code)]

    if row.empty:
        return 'ERROR'

    label = ICD9_LABEL[row['6'].values[0]]
    return label

In [ ]:
# map the data
df_proc['Type'] = df_proc['ICD9_CODE'].progress_map(lambda x: getcode(x))

In [ ]:
ICD9_CODES_DIS = pd.read_csv('ICD9_DIS_CODE.csv')
ICD9_CODES_SE = pd.read_csv('ICD9_SUPP_EXT_CODE.csv')
ICD9_CODES_SF = pd.read_csv('ICD9_SUPP_FACT_CODE.csv')

ICD9_CODES = ICD9_CODES_DIS.append(ICD9_CODES_SE, ignore_index=True)
ICD9_CODES = ICD9_CODES.append(ICD9_CODES_SF, ignore_index=True)
ICD9_CODES = ICD9_CODES.astype(str)

# map the data
df_diag['Type'] = df_diag['ICD9_CODE'].progress_apply(lambda x: getcode(x))

In [ ]:
df_proc.to_csv('mimic-iii-clinical-database-1.4/PROC_LABELD.csv')
df_diag.to_csv('mimic-iii-clinical-database-1.4/DIAG_LABELD.csv')

## Take care of Errors

1. Database where error are given, from the matched
2. Database where the errors are removed

In [ ]:
df_proc = pd.read_csv('mimic-iii-clinical-database-1.4/PROC_LABELD.csv')
df_diag = pd.read_csv('mimic-iii-clinical-database-1.4/DIAG_LABELD.csv')

In [ ]:
# removew errors 
df_proc = df_proc[df_proc['Type'] != 'ERROR']
df_diag = df_diag[df_diag['Type'] != 'ERROR']

# append them togheter
df = df_diag.append(df_proc, ignore_index=True)

In [ ]:
df

In [ ]:
# Filter the errors

In [ ]:
# ... to be continue

In [ ]:
## Issue how to combine mutiple andhow to handle duplicated values

df = df[df['HADM_ID'] == 199993]

## Add code type as columns to rows

In [ ]:
gb = df.groupby('HADM_ID')

df = pd.DataFrame()

for name, group in tqdm(gb):
    
    df_new = pd.DataFrame([np.append(group['ICD9_CODE'], name)], columns=np.append(group['Type'].values, 'HADM_ID')) 
    #print("Init", df_new) 
    
    # remove duplicates
    df_rep = df_new.loc[:,df_new.columns.duplicated()]
    df_new = df_new.loc[:,~df_new.columns.duplicated()]   
    df_copy = df_new
    
    # add duplicates as new rows
    #print("new", df_new)
    #print("rep", df_rep)
    #print("copy", df_copy)
    
    for name, value in df_rep.iteritems():
        df_add = df_copy.copy()
        df_add[name] = value.values[0]
        df_new = df_new.append(df_add, ignore_index=True)
    
    #print("PROD", df_new)
    if df.empty:
        df = df_new
    else:
        df = df.append(df_new, ignore_index=True)
    #print("--- Done ---")

In [ ]:
df

In [ ]:
df.to_csv('mimic-iii-clinical-database-1.4/DIAG_ROWS', index=False)

In [ ]:
df = pd.read_csv('mimic-iii-clinical-database-1.4/DIAG_ROWS')

In [ ]:
# fill in nan
df = df.fillna("NONE")

df